# Рекоммендательные системы
### Дуботолкова Натали, 3530903/70301
### Вариант 7

2.1 Для заданного пользователя (совпадает с вашим номером в таблице) рассчитать оценки для всех фильмов, которые он не оценил.

In [1]:
import numpy as np
import pandas as pd
import json
import math as m
%matplotlib inline

In [2]:
data = pd.read_csv('data.csv', index_col=0)

In [3]:
data.head(7)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,4,1,5,-1,-1,-1,1,3,-1,...,2,4,-1,-1,3,4,-1,2,4,1
User 2,4,5,2,3,3,4,-1,5,3,2,...,-1,2,-1,3,1,5,-1,5,2,2
User 3,4,2,5,4,1,5,-1,3,5,2,...,4,-1,5,4,5,1,2,1,-1,3
User 4,5,-1,2,-1,-1,2,4,3,2,3,...,-1,2,2,2,2,5,3,2,5,5
User 5,4,3,3,3,5,3,3,4,5,2,...,4,5,2,-1,3,5,1,-1,5,-1
User 6,2,-1,3,3,3,3,4,2,-1,1,...,5,-1,1,3,-1,4,4,5,1,2
User 7,4,1,4,4,1,4,5,2,2,-1,...,2,5,1,3,4,-1,4,3,1,4


In [4]:
k = 4
my_user = 6

In [5]:
def sim(u,v):
    sum_uv, sum_u, sum_v = 0, 0, 0
    for i in range (len(u)):
        if u[i]>0 and v[i]>0:
                sum_u += u[i]**2
                sum_v += v[i]**2
                sum_uv += v[i]*u[i]             
    return round((sum_uv/(m.sqrt(sum_u)*m.sqrt(sum_v))),3)
        

In [6]:
all_similarities = np.zeros((40, 40))

In [7]:
for i in range(data.shape[0]):
    listOfU = list(data.iloc[i])
    j = i+1
    for j in range(j, data.shape[0]):
        listOfV = list(data.iloc[j])
        all_similarities[i, j] = sim(listOfU, listOfV)
        all_similarities[j, i] = all_similarities[i, j]

In [8]:
all_similarities #матрица сходств всех пользователей

array([[0.   , 0.79 , 0.741, ..., 0.891, 0.745, 0.83 ],
       [0.79 , 0.   , 0.742, ..., 0.852, 0.766, 0.835],
       [0.741, 0.742, 0.   , ..., 0.812, 0.806, 0.729],
       ...,
       [0.891, 0.852, 0.812, ..., 0.   , 0.809, 0.886],
       [0.745, 0.766, 0.806, ..., 0.809, 0.   , 0.799],
       [0.83 , 0.835, 0.729, ..., 0.886, 0.799, 0.   ]])

In [9]:
my_similarities = all_similarities[my_user] #матрица сходств для моего пользователя (7-го)

In [10]:
my_similarities

array([0.838, 0.823, 0.854, 0.833, 0.831, 0.882, 0.   , 0.849, 0.743,
       0.796, 0.755, 0.896, 0.843, 0.774, 0.769, 0.862, 0.773, 0.848,
       0.886, 0.8  , 0.914, 0.802, 0.865, 0.805, 0.829, 0.802, 0.828,
       0.824, 0.89 , 0.898, 0.862, 0.871, 0.807, 0.883, 0.82 , 0.844,
       0.828, 0.871, 0.795, 0.866])

In [11]:
my_LikeMinded = my_similarities.argsort()[-k:]
my_LikeMinded #массив индексов четырёх людей, у кого со мной наибольшее сходство

array([28, 11, 29, 20], dtype=int32)

In [12]:
#средняя оценка пользователя u
def average(u):
    sum_u = 0
    counter = 0
    for i in range (len(u)):
        if u[i]>0:
            counter += 1
            sum_u += u[i]
    return round((sum_u/counter),3)

In [13]:
#находим фильмы (их индексы), которые пользователь ещё не смотрел
duser = data.iloc[my_user]
user_films = []
for index, i in enumerate(duser):
    if i<0:
        user_films.append(index)
user_films    

[9, 11, 16, 17, 25]

In [14]:
# расчёт оценки на конкретный фильм
def r_ui(film):
    listOfU = list(data.iloc[my_user])
    r_u = average(listOfU)
    sum_up, sum_down = 0, 0
    for v in my_LikeMinded:
        if(data.iloc[v,film]>0):
            listOfV = list(data.iloc[v])
            _sim = my_similarities[v]
            sum_up += round(_sim*(data.iloc[v,film]-average(listOfV)),3)
            sum_down += abs(_sim)
    return round((r_u + sum_up/sum_down),3)

In [15]:
listOfU = list(data.iloc[my_user])
listRes = {}
# находим индексы непросматриваемых фильмов и пытаемся предсказать оценку
for idx, num in enumerate(listOfU):
        if num<0:
            listRes['movie ' + str(idx+1) + ': '] = r_ui(idx)
listRes

{'movie 10: ': 2.14,
 'movie 12: ': 3.396,
 'movie 17: ': 1.288,
 'movie 18: ': 3.214,
 'movie 26: ': 4.639}

2. Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных.

Я бы вообще не советовала ему ничего смотреть - у него слишком низкий средний балл, (~2,3) Так что высока вероятность, что предложенные фильмы ему тоже не понравятся.
Однако, попробуем посмотреть, какие фильмы смотрели по выходным дома те 4 человека, которые имеют с ним +- такой же вкус и выберем фильм с наибольшим рейтингом.

In [16]:
days = pd.read_csv("context_day.csv", index_col=0)
places = pd.read_csv("context_place.csv", index_col=0)

In [17]:
days.head(5)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,Thu,Tue,Mon,-1,-1,-1,Tue,Sun,-1,...,Thu,Thu,-1,-1,Sun,Sat,-1,Wed,Wed,Thu
User 2,Mon,Wed,Mon,Sat,Sat,Thu,-1,Mon,Fri,Thu,...,-1,Sun,-1,Mon,Mon,Fri,-1,Sun,Tue,Wed
User 3,Tue,Wed,Sat,Tue,Sat,Mon,-1,Thu,Wed,Thu,...,Tue,-1,Wed,Thu,Tue,Thu,Wed,Wed,-1,Sun
User 4,Mon,-1,Wed,-1,-1,Sun,Sat,Mon,Mon,Wed,...,-1,Sun,Fri,Sat,Thu,Sat,Fri,Fri,Sat,Sun
User 5,Thu,Fri,Wed,Fri,Mon,Thu,Sat,Tue,Thu,Fri,...,Fri,Fri,Mon,-1,Wed,Mon,Sat,-1,Wed,-1


In [18]:
places.head(5)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
,,,,,,,,,,,,,,,,,,,,,
User 1,-1,h,h,c,-1,-1,-1,v,h,-1,...,h,c,-1,-1,c,c,-1,v,h,c
User 2,v,v,v,v,h,c,-1,v,h,c,...,-1,c,-1,c,h,v,-1,v,v,c
User 3,v,h,c,c,c,c,-1,h,c,v,...,v,-1,v,v,h,h,h,v,-1,v
User 4,v,-1,h,-1,-1,h,v,v,c,h,...,-1,v,h,h,c,v,c,v,v,c
User 5,c,c,c,c,v,h,v,v,c,h,...,v,c,v,-1,h,c,v,-1,h,-1


In [19]:
#находим фильмы (их индексы), которые пользователь ещё не смотрел
duser = data.iloc[my_user]
user_films = []
for index, i in enumerate(duser):
    if i<0:
        user_films.append(index)
user_films

[9, 11, 16, 17, 25]

In [20]:
# нашла фильмы, которые наша четверка смотрела по выходным

allFilmsOnWeekend = []
for user in my_LikeMinded:
    for idx,el in enumerate(days.iloc[user]):
        if (el.strip() == 'Sun' or el.strip() == 'Sat'):
            allFilmsOnWeekend.append(idx+1)
auxiliaryList = list(set(allFilmsOnWeekend))
print(auxiliaryList)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 24, 25, 27, 28, 29]


In [21]:
#нашли совпадения в этих двух списках
films = list(set(user_films) & set(auxiliaryList))
print(set(user_films))
print(set(auxiliaryList))
print(films)

{9, 11, 16, 17, 25}
{1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 24, 25, 27, 28, 29}
[16, 9, 17, 25]


In [22]:
# нашла фильмы, которые наша четверка смотрела дома
allFilmsAtHome = []
for user in my_LikeMinded:
    for idx,el in enumerate(places.iloc[user]):
        if el.strip() == 'h':
            allFilmsAtHome.append(idx+1)   
auxiliaryList = list(set(allFilmsOnWeekend))
print(auxiliaryList)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 24, 25, 27, 28, 29]


In [23]:
#нашли совпадения в этих двух списках
films = list(set(films) & set(auxiliaryList))
print(films)

[16, 9, 17, 25]


In [24]:
# посчитала рейтинг каждого фильма, среди единомышленников
count = 0
raiting_film = {}
temp = 0

for i in films:
    for j in my_LikeMinded:
        raiting = data.iloc[j][i]
#         print(i, raiting)
        if raiting > 0:
            temp += 1
            count+= raiting
    raiting_film[i] = round(count / temp, 3)

raiting_film

{16: 1.667, 9: 2.143, 17: 2.6, 25: 3.286}

In [25]:
#нашла максимальное значение среди всех фильмов
max_val = max(raiting_film.values())
dictRes2 = {}
recom_film = {f'movie {k+1}: ':v for k, v in raiting_film.items() if v == max_val}
print(recom_film)
#Ставим предположительную оценку из тех, что я предстказала ранее
for key1, value1 in listRes.items():
    for key2, value2 in recom_film.items():
        if key2 == key1:
            recom_film[key2] = value1
print(recom_film)

{'movie 26: ': 3.286}
{'movie 26: ': 4.639}


In [26]:
result = {"user": 7, "1": listRes, "2": recom_film}
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)